# 대화 요약을 위한 지도 학습 모델(Instructor-Model) 미세 조정하기

## 준비사항
아래 실습은 AWS ml.m5.2xlarge 인스턴스에서 수행했습니다.

## 커널 및 필요한 종속성 설정하기

In [ ]:
%pip install -U --disable-pip-version-check \
    torch==2.3.1 \
    transformers==4.44.0 \
    datasets==2.21.0 \
    accelerate==0.33.0 \
    evaluate==0.4.2 \
    py7zr==0.22.0 \
    sentencepiece==0.2.0 \
    rouge-score==0.1.2 \
    loralib==0.1.2 \
    peft==0.12.0 \
    trl==0.9.6 \
    ipywidgets==8.1.3

In [ ]:
model_checkpoint='google/flan-t5-base'

In [ ]:
# 이 디렉토리는 이전 노트북에서 생성되었습니다.
local_data_processed_path = './data-summarization-processed/'

# 패키지 적재

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, TrainingArguments, Trainer, GenerationConfig
from datasets import load_dataset
import datasets
import torch
import time
import evaluate
import numpy as np
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

# 허깅페이스 모델 적재

사전 훈련된 Flan-T5 모델을 허깅페이스에서 직접 불러올 수 있습니다. 이번 실습에선 Flan 기본 버전을 사용합니다. 이 모델 버전은 약 2억 4,700만 개의 모델 파라미터를 가지고 있어 다른 대형 언어 모델(LLM)과 비교해 상대적으로 작습니다. 더 높은 품질의 결과를 원한다면 이 모델의 더 큰 버전들을 고려해 보시는 것이 좋습니다.

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

In [ ]:
params = sum(p.numel() for p in model.parameters())
print(f'모델 파라미터의 총 수: {params}')

# 처리된 데이터 적재

# 데이터 세트 적재

미리 처리된 `DialogSum` 데이터 세트는 로컬 디렉토리에서 직접 불러올 수 있습니다. 이 데이터 세트에는 약 15,000개의 대화 예제와 그에 대한 인간 요약본이 포함되어 있습니다.

In [ ]:
tokenized_dataset = load_dataset(
    local_data_processed_path,
    data_files={'train': 'train/*.parquet', 'test': 'test/*.parquet', 'validation': 'validation/*.parquet'}
).with_format("torch")
tokenized_dataset

# 미세 조정 전에 제로샷 프롬프트로 모델 테스트하기

아래 예제에서는 데이터 세트에 제공된 기본 요약과 비교하여 모델의 요약 기능이 부족함을 강조합니다. 모델이 기본 요약과 비교했을 때 대화를 요약하는 데 어려움을 겪지만, 일부 중요한 정보를 텍스트에서 추출합니다. 이는 모델이 주어진 작업에 맞게 미세 조정될 수 있음을 시사합니다.

In [ ]:
idx = 2
diag = tokenizer.decode(tokenized_dataset['test'][idx]['input_ids'], skip_special_tokens=True)
model_input = tokenizer(diag, return_tensors="pt").input_ids
summary = tokenizer.decode(tokenized_dataset['test'][idx]['labels'], skip_special_tokens=True)

original_outputs = model.to('cpu').generate(model_input, GenerationConfig(max_new_tokens=200))
original_text_output = tokenizer.decode(original_outputs[0], skip_special_tokens=True)

diag_print = diag.replace(' #',' \n#')
print(f"프롬프트:\n--------------------------\n{diag_print}\n--------------------------")
print(f'\n원본 모델 응답: {original_text_output}')
print(f'기본 요약 : {summary}')

# 지도 학습 모델 미세 조정하기

데이터 세트를 전처리했으니 허깅페이스 내장 `Trainer` 클래스를 사용하여 주어진 작업에 맞게 모델을 미세 조정할 수 있습니다. 이 전체 모델을 훈련하는 데는 GPU에서 몇 시간이 걸리므로 시간을 절약하기 위해 다운 샘플링 없이 10번의 에포크 동안 훈련된 모델의 체크포인트가 제공됩니다. 직접 모델을 완전히 훈련해보고 싶다면 코드를 변경하는 방법에 대한 인라인 주석을 참고하세요. GPU 머신에서 훈련하려는 경우 체크포인트 제공을 시작점으로 `ml.g5.xlarge` 인스턴스를 사용했습니다.

In [ ]:
# 실습 시간 절약을 위해, 데이터 세트 부분 샘플링
# 모델을 완전히 훈련시키고 싶다면, 부분 샘플링을 수정하여 더 큰 데이터 세트를 생성하는게 좋습니다.
sample_tokenized_dataset = tokenized_dataset.filter(lambda example, indice: indice % 100 == 0, with_indices=True)

output_dir = f'./diag-summary-training-{str(int(time.time()))}'
training_args = TrainingArguments(
    output_dir=output_dir,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=1e-5,
    num_train_epochs=1,
    # num_train_epochs=10, # 실험할 시간이 더 많을 때는 더 많은 에포크 수를 사용하세요.
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    weight_decay=0.01,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=sample_tokenized_dataset['train'],
    eval_dataset=sample_tokenized_dataset['validation']
)

In [ ]:
trainer.train()

# 훈련된 모델과 원본 모델 적재

모델 훈련이 완료되면 허깅페이스의 원본 모델과 미세 조정된 모델을 불러와 질적 및 양적 비교를 수행합니다.

In [ ]:
!aws s3 cp --recursive s3://dsoaws/models/flan-dialogue-summary-checkpoint/ ./flan-dialogue-summary-checkpoint/

In [ ]:
# 직접 모델을 훈련하고 우리의 모델과 비교해 보고 싶다면, 아래 코드를 변경하세요.
# 여러분의 체크포인트 디렉토리로 변경하세요.

supervised_fine_tuned_model_path = "./flan-dialogue-summary-checkpoint"
# supervised_fine_tuned_model_path = f"./{output_dir}/<여러분의-체크포인트-디렉토리를-입력하세요>"

tuned_model = AutoModelForSeq2SeqLM.from_pretrained(supervised_fine_tuned_model_path)
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

In [ ]:
%store supervised_fine_tuned_model_path

# 미세 조정 후 제로샷 추론으로 질적 결과 평가하기

많은 생성형 AI 응용 프로그램에서 "내 모델이 예상대로 작동하고 있는가?"라는 질문을 스스로 던지는 질적 접근은 좋은 시작점입니다. 아래 예제(이 노트북에서 처음 시작한 것과 동일한 예제)에서는 미세 조정된 모델이 대화를 합리적으로 요약할 수 있음을 보여줍니다. 이는 모델이 질문에 대해 이해하지 못했던 원래의 상태와 비교됩니다.

In [ ]:
idx = 2
diag = tokenizer.decode(tokenized_dataset['test'][idx]['input_ids'], skip_special_tokens=True)
model_input = tokenizer(diag, return_tensors="pt").input_ids
summary = tokenizer.decode(tokenized_dataset['test'][idx]['labels'], skip_special_tokens=True)

original_outputs = model.to('cpu').generate(
    model_input,
    GenerationConfig(max_new_tokens=200, num_beams=1),
)
outputs = tuned_model.to('cpu').generate(
    model_input,
    GenerationConfig(max_new_tokens=200, num_beams=1,),
)
text_output = tokenizer.decode(outputs[0], skip_special_tokens=True)

diag_print = diag.replace(' #',' \n#')
print(f"프롬프트:\n--------------------------\n{diag_print}\n--------------------------")
print(f'Flan-T5 응답: {original_text_output}')
print(f'지도 학습된 응답 (Flan-T5 기반): {text_output}')
print(f'원본 데이터 세트의 기본 요약: {summary}')

# ROUGE 메트릭을 사용한 양적 결과 평가

[ROUGE 메트릭](https://en.wikipedia.org/wiki/ROUGE_(metric))은 모델이 생성한 요약의 유효성을 수치화하는 데 도움됩니다. 이 메트릭은 일반적으로 사람이 작성한 "기준" 요약과 모델의 요약을 비교합니다. 완벽하지는 않지만 미세 조정을 통해 달성한 요약 효과의 전반적인 증가를 나타냅니다.

In [ ]:
rouge = evaluate.load('rouge')

## 요약의 하위 섹션 평가

In [ ]:
# 시간을 절약하기 위해 각 모델로 몇 개의 요약만 생서합니다.
# 실습실 외부에서는 생성할 검증 요약의 수를 늘려보는 것이 좋습니다.
dialogues = tokenized_dataset['test'][0:10]['input_ids']
baseline_summaries = tokenized_dataset['test'][0:10]['labels']

# 원본 요약 디코딩
human_baseline_summaries = []
for base_summary in baseline_summaries:
    human_baseline_summaries.append(tokenizer.decode(base_summary, skip_special_tokens=True))

# 요약 생성
original_outputs = model.generate(dialogues, GenerationConfig(max_new_tokens=200))
tuned_outputs = tuned_model.generate(dialogues, GenerationConfig(max_new_tokens=200))

In [ ]:
# 요약을 리스트에 저장
original_model_summaries = []
tuned_model_summaries = []

# 모든 요약 디코딩
for original_summary, tuned_summary in zip(original_outputs, tuned_outputs):
    original_model_summaries.append(tokenizer.decode(original_summary, skip_special_tokens=True))
    tuned_model_summaries.append(tokenizer.decode(tuned_summary, skip_special_tokens=True))

In [ ]:
original_results = rouge.compute(
    predictions=original_model_summaries,
    references=human_baseline_summaries,
    use_aggregator=True,
    use_stemmer=True,
)

In [ ]:
tuned_results = rouge.compute(
    predictions=tuned_model_summaries,
    references=human_baseline_summaries,
    use_aggregator=True,
    use_stemmer=True,
)

In [ ]:
original_results

In [ ]:
tuned_results

## 전체 데이터셋 평가

"diag-summary-training-results.csv" 파일에는 모든 모델 결과가 미리 채워진 목록이 포함되어 있어 더 큰 데이터 섹션을 평가하는 데 사용할 수 있습니다. 결과는 모든 ROUGE 메트릭에서 상당히 개선된 것을 보여줍니다!

In [ ]:
import pandas as pd
results = pd.read_csv("diag-summary-training-results.csv")
original_model_summaries = results['original_model_summaries'].values
tuned_model_summaries = results['tuned_model_summaries'].values
human_baseline_summaries = results['human_baseline_summaries'].values

In [ ]:
original_results = rouge.compute(
    predictions=original_model_summaries,
    references=human_baseline_summaries[0:len(original_model_summaries)],
    use_aggregator=True,
    use_stemmer=True,
)

In [ ]:
tuned_results = rouge.compute(
    predictions=tuned_model_summaries,
    references=human_baseline_summaries[0:len(tuned_model_summaries)],
    use_aggregator=True,
    use_stemmer=True,
)

In [ ]:
original_results

In [ ]:
tuned_results

In [ ]:
improvement = (np.array(list(tuned_results.values())) - np.array(list(original_results.values())))
for key, value in zip(tuned_results.keys(), improvement):
    print(f'{key} 명령어 기반 미세 조정 후 절대 개선 비율: {value*100:.2f}%')